In [2]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from typing import List, Union



# 1. Defina as features (variáveis de entrada) na ordem correta


In [10]:
FEATURE_NAMES = [
    'COD_UF_EMIT', 
    'TIP_FIN_NFE', 
    'COD_CEST', 
    'COD_CST', 
    'COD_NCM', 
    'COD_CFOP', 
    'EMIT_COD_CNAE', 
    'EMIT_CRT', 
    'EMIT_IND_SN', 
    'DEST_CNAE_PRINC', 
    'DEST_POSSUI_IE', 
    'DEST_SIMPLES'
]

# 2. **NOVAS VARIÁVEIS** - Identifique quais features são Categóricas
# Baseado na descrição das features, estas são as mais prováveis de serem categóricas
CATEGORICAL_FEATURES = [
    'COD_UF_EMIT', 
    'TIP_FIN_NFE', 
    'COD_CEST', 
    'COD_CST', 
    'COD_NCM', 
    'COD_CFOP', 
    'EMIT_COD_CNAE', 
    'EMIT_CRT', 
    'EMIT_IND_SN', 
    'DEST_CNAE_PRINC', 
    'DEST_POSSUI_IE', 
    'DEST_SIMPLES'
]

# 2. Caminho do arquivo do modelo


In [4]:
MODEL_FILE_PATH = "modelo_lgbm 1.txt"


# 3. Função principal para carregar o modelo e fazer a predição


In [11]:
def carregar_e_classificar(dados_x: pd.DataFrame) -> Union[np.ndarray, int]:
    """
    Carrega o modelo LightGBM, realiza o pré-processamento necessário
    e faz a classificação dos dados fornecidos.
    """
    try:
        # Carrega o modelo (Booster)
        bst = lgb.Booster(model_file=MODEL_FILE_PATH)
    except lgb.basic.LightGBMError as e:
        print(f"Erro ao carregar o modelo: {e}")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None

    # Garante que as colunas do DataFrame estão na ordem correta
    dados_para_predicao = dados_x[FEATURE_NAMES].copy()

    # **ETAPA DE CORREÇÃO:** Converte as colunas categóricas para o tipo 'category'
    for col in CATEGORICAL_FEATURES:
        if col in dados_para_predicao.columns:
            # Converte para string antes de category para tratar valores mistos (num/str)
            dados_para_predicao[col] = dados_para_predicao[col].astype(str).astype('category')
    
    # Realiza a predição. O LightGBM agora deve reconhecer as colunas categóricas.
    probabilidades = bst.predict(dados_para_predicao)

    # A classificação final é a classe com a maior probabilidade.
    classificacao_y = np.argmax(probabilidades, axis=1)

    # Retorna o valor ou a lista de valores
    if len(classificacao_y) == 1:
        return classificacao_y[0]
    return classificacao_y

# 4. Exemplo de Uso


In [15]:
# --- CRIE SEUS DADOS DE ENTRADA AQUI ---
# Valores de exemplo que requerem a tipagem categórica
dados_exemplo = pd.DataFrame({
    'COD_UF_EMIT': ["PE", "SP", "PE"], 
    'TIP_FIN_NFE': [1, 1, 1],
    'COD_CEST': [1003000, 2100100, None],
    'COD_CST': [0, 0, 10],
    'COD_NCM': [69072100, 73211100, 23099010],
    'COD_CFOP': [6107, 6107, 6401],
    'EMIT_COD_CNAE': [None, 2751100, None], # Valores numéricos
    'EMIT_CRT': [3, 3, 3],
    'EMIT_IND_SN': ['N', 'N', 'N'],
    'DEST_CNAE_PRINC': [None, None, 4789004], # Valores numéricos
    'DEST_POSSUI_IE': ["S", "N", "S"],
    'DEST_SIMPLES': ['N', 'S', 'S']
})

print("--- Dados de Entrada (X) ---")
print(dados_exemplo)
print("\n")

# Realiza a classificação
resultado_y = carregar_e_classificar(dados_exemplo)

if resultado_y is not None:
    print("--- Resultado da Classificação (Y) ---")
    if isinstance(resultado_y, int):
            print(f"Classificação prevista: {resultado_y}")
    else:
        dados_exemplo['CLASSIFICACAO_Y'] = resultado_y
        print(dados_exemplo[['CLASSIFICACAO_Y']])
        print(f"\nResultado completo (array): {resultado_y}")

--- Dados de Entrada (X) ---
  COD_UF_EMIT  TIP_FIN_NFE   COD_CEST  COD_CST   COD_NCM  COD_CFOP  \
0          PE            1  1003000.0        0  69072100      6107   
1          SP            1  2100100.0        0  73211100      6107   
2          PE            1        NaN       10  23099010      6401   

   EMIT_COD_CNAE  EMIT_CRT EMIT_IND_SN  DEST_CNAE_PRINC DEST_POSSUI_IE  \
0            NaN         3           N              NaN              S   
1      2751100.0         3           N              NaN              N   
2            NaN         3           N        4789004.0              S   

  DEST_SIMPLES  
0            N  
1            S  
2            S  


--- Resultado da Classificação (Y) ---
   CLASSIFICACAO_Y
0                2
1                2
2                2

Resultado completo (array): [2 2 2]
